# Data Preprocessing

## STEP-1 Tokenization

In [5]:
import nltk
import re
# download both punkt and punkt_tab (needed in latest versions)
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
# Download the required POS tagger model if not already present
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker')
nltk.download('maxent_ne_chunker_tab')
nltk.download('words')
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AMAN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\AMAN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\AMAN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\AMAN\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\AMAN\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker_tab to
[nltk_data]     C:\Users\AMAN\AppData\Roaming\nltk_data...
[nltk_data]   Package ma

In [6]:
# Fetching the data
with open(r"C:\Users\AMAN\Downloads\DATA Scientist\GitHub\NLP Learning\Learning\the-verdict.txt","r", encoding="utf-8") as f:
    raw_text = f.read()
print("Total no. of character: ", len(raw_text))
print(raw_text[:99])

Total no. of character:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [7]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',raw_text)
preprocessed = [item for item in preprocessed if item.strip()]
print(preprocessed[:25])
print(len(preprocessed))

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me']
4690


## STEP-2 Creating Token-ID's

In [8]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [9]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [10]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i>=50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [11]:
# Creating class for Encoder and Decoder
class SimpleTokenizerV1:
    def __init__(self,vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self,text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        # Handle unknown tokens gracefully: skip tokens not in vocab
        ids = [self.str_to_int[s] for s in preprocessed if s in self.str_to_int]
        return ids
    
    def decode(self,ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replacing spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])',r'\1',text)
        return text


In [12]:
# Testing the class- Encoder and Decoder on the Sample Text
tokenizer = SimpleTokenizerV1(vocab=vocab)
text = """ " My dear, since I've chucked painting people don't say that stuff about me--they say it about Victor Grindle,"
 was his only protest, as he rose from the table and strolled out onto the sunlit terrace."""
ids = tokenizer.encode(text)
print(ids)


[1, 68, 319, 5, 895, 53, 2, 1062, 261, 748, 763, 359, 2, 970, 856, 987, 945, 118, 663, 6, 994, 856, 585, 118, 105, 42, 5, 1, 1077, 549, 731, 800, 5, 177, 533, 846, 477, 988, 971, 157, 940, 738, 732, 988, 956, 983, 7]


In [13]:
# Testing decode
print(tokenizer.decode(ids))

" My dear, since I' ve chucked painting people don' t say that stuff about me -- they say it about Victor Grindle," was his only protest, as he rose from the table and strolled out onto the sunlit terrace.


In [14]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(['<|endoftext|>','<|unknown|>'])
vocab = {token: integer for integer, token in enumerate(all_tokens)}

In [15]:
class SimpleTokenizerV2:
    def __init__(self,vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self,text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else '<|unknown|>'
            for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self,ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replacing spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])',r'\1',text)
        return text


In [16]:
tokenizer = SimpleTokenizerV2(vocab=vocab)
text1 = "Hello, do you like tea."
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1,text2))
print(text)

Hello, do you like tea. <|endoftext|> In the sunlit terraces of the palace.


In [17]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 7, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [18]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

In [19]:
text = (
    "Hello, my name is Vansh. <|endoftext|> I am learning LLMS"
)
integers = tokenizer.encode(text,allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 616, 1438, 318, 569, 504, 71, 13, 220, 50256, 314, 716, 4673, 27140, 5653]


#   Creating Input Target Pairs

In [20]:
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [21]:
enc_sample = enc_text[50:]

In [22]:
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y: {y}")

x: [290, 4920, 2241, 287]
y: [4920, 2241, 287, 257]


In [23]:
for i in range(1,context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context,"----->",desired)

[290] -----> 4920
[290, 4920] -----> 2241
[290, 4920, 2241] -----> 287
[290, 4920, 2241, 287] -----> 257


In [24]:
for i in range(1,context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context),"----->",tokenizer.decode([desired]))

 and ----->  established
 and established ----->  himself
 and established himself ----->  in
 and established himself in ----->  a


In [25]:
import torch
from torch.utils.data import Dataset,DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self,txt,tokenizer,max_length,stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire set
        token_ids = tokenizer.encode(txt,allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences
        for i in range(0,len(token_ids)-max_length,stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self,index):
      return self.input_ids[index],self.target_ids[index]

In [26]:

def create_dataloaderV1(txt,batch_size=4,max_length=256,stride=128,shuffle=True,drop_last=True,num_workers=0):
    # Intializing the tokenizer 
    tokenizer = tiktoken.get_encoding("gpt2")

    # Creating dataset
    dataset = GPTDatasetV1(txt,tokenizer,max_length,stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size = batch_size,
        shuffle = shuffle,
        drop_last = drop_last,
        num_workers = num_workers
    )

    return dataloader

In [27]:
dataloader = create_dataloaderV1(raw_text,batch_size=1,max_length=4,stride=1,shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [28]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [34]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size,output_dim)

In [ ]:
max_length = 4
dataloader = create_dataloaderV1(
    raw_text,batch_size=8,max_length=max_length,stride=max_length,shuffle=False)
data_iter = iter(dataloader)
inputs,targets = next(data_iter)

In [33]:
print("Token IDs: \n",inputs)
print("\n Input Shape: \n",inputs.shape)

Token IDs: 
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

 Input Shape: 
 torch.Size([8, 4])


In [35]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [37]:
# Creating Postional Embedding Layer
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length,output_dim)
pos_embedding = pos_embedding_layer(torch.arange(max_length))
print(pos_embedding.shape)

torch.Size([4, 256])


In [38]:
input_embeddings = token_embeddings + pos_embedding
print(input_embeddings.shape)

torch.Size([8, 4, 256])
